### Sample dashboard to Visualize Seasonal Forecast Data as Box plot -  March 2023 -  JRC

#### Run the below line to launch the GUI..

In [ ]:
exec("""\nfrom ipyleaflet import Map, DrawControl, GeoJSON, LayersControl, SearchControl, TileLayer\nfrom ipywidgets import widgets, Layout\nfrom IPython.display import display, HTML\nfrom datetime import datetime\nimport dateutil.relativedelta\nfrom apps.productmanagement.extended_Dataset import ExtDataset\nimport xarray as xr\n\ndmax = datetime.today()\ndmin = dmax + dateutil.relativedelta.relativedelta(months=-3)\n\ndatemax= widgets.DatePicker(value=dmax,description='Max date:',\n                             style={'description_width': 'initial'},indent=False,layout=widgets.Layout(width='200px', height='32px'))\ndatemin = widgets.DatePicker(value=dmin,description='Min date:',\n                             style={'description_width': 'initial'},indent=False,layout=widgets.Layout(width='200px', height='32px'))\ndropdown = widgets.Dropdown(description='Products:', options=['s51-monthly-tp','s51-monthly-2mt','s51-monthly-sst'])\nbutton = widgets.Button(\n    description='List the files',\n    disabled=False,\n    button_style='', # 'success', 'info', 'warning', 'danger' or ''\n    tooltip='List the files for the selected product and dates',\n    icon='check' # (FontAwesome names without the `fa-` prefix)\n)\n\nproducts = widgets.Select(description='Files', options=[],rows=20, layout=Layout(width='1000px', min_width='200px', height='108px'))\n\nclear_button = widgets.Button(\n    description='Clear List',\n    disabled=False,\n    button_style='', # 'success', 'info', 'warning', 'danger' or ''\n    tooltip='Clear the output in the display',\n    icon='check' # (FontAwesome names without the `fa-` prefix)\n)\n\nlist_dimension_button = widgets.Button(\n    description='List Dimenions',\n    disabled=False,\n    button_style='', # 'success', 'info', 'warning', 'danger' or ''\n    tooltip='List Dimenions',\n    icon='check' # (FontAwesome names without the `fa-` prefix)\n)\n\noutput = widgets.Output()\nDimension_output = widgets.Output()\nview_output = widgets.Output()\ndisplay(datemin,datemax,dropdown, button, products,output, Dimension_output,view_output)\n\ntime = widgets.Dropdown(description='Forecast time', options=[])\n#number = widgets.Dropdown(description='Ensemble member',options=[])\nnumber = widgets.SelectMultiple(\n    options=[],\n    #value=['Oranges'],\n    #rows=10,\n    description='Ensemble member',\n    disabled=False\n)\n\ndef on_button_clicked(b):\n    product= dropdown.value\n    sprod = dropdown.value.split("-")[-1]\n    kwargs = {\n            'product_code': product,\n            'version': "1.0",\n            'sub_product_code': sprod,\n            'mapset': 'CDS-ACP-1deg'\n        }\n    tp = ExtDataset(**kwargs)\n    dmin = datemin.value.strftime('%Y%m%d')\n    dmax = datemax.value.strftime('%Y%m%d')\n    tp.filter(str(dmin),str(dmax))\n    files = tp.get_filenames_range()\n    products.options = files\n    #display(clear_button)\n    with output:\n        #print(dmin+"-->"+dmax+ dropdown.value)\n        #print(list(files))\n        output.clear_output()\n        display(clear_button, lat_widget, lon_widget, plot_Spaghetti_button)\nbutton.on_click(on_button_clicked)\n\ndef on_button_clear(b):\n    products.options = []\n    with output:\n        Dimension_output.clear_output()\n        \nclear_button.on_click(on_button_clear)\n\n\nlat_widget = widgets.BoundedFloatText(\n    value=0,\n    min=-45,\n    max=45,\n    step=1,\n    description='Lat:',\n    disabled=False\n)\nlon_widget = widgets.BoundedFloatText(\n    value=0,\n    min=-180,\n    max=180,\n    step=1,\n    description='Long:',\n    disabled=False\n)\nplot_Spaghetti_button = widgets.Button(\n    description='Box plot',\n    disabled=False,\n    button_style='', # 'success', 'info', 'warning', 'danger' or ''\n    tooltip='Box plot',\n    icon='check' # (FontAwesome names without the `fa-` prefix)\n)\n\ndef on_list_dimension(b):\n    time.options=[]\n    number.options=[]\n    product= dropdown.value\n    sprod = dropdown.value.split("-")[-1]\n    kwargs = {\n            'product_code': product,\n            'version': "1.0",\n            'sub_product_code': sprod,\n            'mapset': 'CDS-ACP-1deg'\n        }\n    extD = ExtDataset(**kwargs)\n    extD.assign_dims_values(products.value)\n    # time = widgets.Dropdown(options=tp.dimensions.get('time').value)\n    # number = widgets.Dropdown(options=tp.dimensions.get('number').value)\n    time.options=extD.dimensions.get('time').value\n    number.options=extD.dimensions.get('number').value\n    \n    with Dimension_output:\n        print(b)\n        Dimension_output.clear_output()\n        display(lat_widget, lon_widget, plot_Spaghetti_button)\n\nlist_dimension_button.on_click(on_list_dimension)\n\n# def get_data():\n#     data = []\n#     for time in ds[extD.data_variable].loc[extD.dimensions.get('time').value: data.append(ds[extD.data_variable].loc[time,:,lat_widget.value,lon_widget.value])\n#     return data\n\ndef on_plot_Spaghetti(b):\n    import matplotlib.pyplot as plt\n    product= dropdown.value\n    sprod = dropdown.value.split("-")[-1]\n    kwargs = {\n            'product_code': product,\n            'version': "1.0",\n            'sub_product_code': sprod,\n            'mapset': 'CDS-ACP-1deg'\n        }\n    extD = ExtDataset(**kwargs)\n    extD.assign_dims_values(products.value)\n    ds = xr.open_dataset(products.value)\n    \n    with view_output:\n        view_output.clear_output()\n        fig = plt.figure(figsize =(10, 7))\n        ax = fig.add_subplot(111)\n        ax.boxplot([ds[extD.data_variable].loc[extD.dimensions.get('time').value[0],:,lat_widget.value,lon_widget.value], \n            ds[extD.data_variable].loc[extD.dimensions.get('time').value[1],:,lat_widget.value,lon_widget.value],\n            ds[extD.data_variable].loc[extD.dimensions.get('time').value[2],:,lat_widget.value,lon_widget.value],\n            ds[extD.data_variable].loc[extD.dimensions.get('time').value[3],:,lat_widget.value,lon_widget.value],\n            ds[extD.data_variable].loc[extD.dimensions.get('time').value[4],:,lat_widget.value,lon_widget.value],\n            ds[extD.data_variable].loc[extD.dimensions.get('time').value[5],:,lat_widget.value,lon_widget.value]])\n        # x-axis labels\n        ax.set_xticklabels(['Lead month 1', 'Lead month 2', 'Lead month 3', 'Lead month 4','Lead month 5','Lead month 6'])\n        ax.set_ylabel(ds[extD.data_variable].long_name+' ('+ds[extD.data_variable].units+')')\n        plt.show()\n\n    \n    #data = get_data()\n    # for time in ds[extD.data_variable].loc[extD.dimensions.get('time').value:\n    #     data.append(ds[extD.data_variable].loc[time,:,lat_widget.value,lon_widget.value])\n                                           \n    # pyplot.boxplot(data)\n    # pyplot.show()\n    #ds[extD.data_variable].sel(number=extD.dimensions.get('number').value,longitude=lon_widget.value, latitude=lat_widget.value).plot.line(x="time")\n\n\nplot_Spaghetti_button.on_click(on_plot_Spaghetti)\n""")